* MSV_HoVaTen:
521H0501_Phan Minh Hoàng,
521H0398_Lê Nguyễn Viết Hiệp,
521H0276_Trang Sĩ Nam



In [598]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [599]:
data = pd.read_csv('bike_buyers.csv')
print(data.shape)

(1000, 13)


In [600]:
data = data.dropna()

In [601]:
# Check duplicate
data.duplicated().value_counts()

False    952
Name: count, dtype: int64

In [602]:
data = data.drop_duplicates()
data.shape

(952, 13)

* Transformation

In [603]:
# Tranform data
from sklearn.preprocessing import LabelEncoder

types = data.dtypes
names = list(data.columns)

le = LabelEncoder()
for i in range(len(types)):
    if types.iloc[i]=='object':
        le.fit_transform(data[names[i]])
        data[names[i]] = le.transform(data[names[i]])

* Normalization


In [604]:
from sklearn.preprocessing import MinMaxScaler
features = data.drop(['Purchased Bike'], axis=1)

scaler = MinMaxScaler()
model=scaler.fit(features)
scaled_data= pd.DataFrame(model.transform(features))

* Feature Selection

In [605]:
# split data
X = scaled_data
y = data['Purchased Bike']

In [606]:
from sklearn.feature_selection import SelectKBest, chi2, f_regression
select_feature =  SelectKBest(score_func=chi2, k = 8)
f_best = select_feature.fit(X, y)               
X_new = f_best.transform(X)
X_new.shape

(952, 8)

* FFNN

In [607]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

X_train, X_test, Y_train, Y_test = train_test_split(X_new, y, random_state = 42, test_size=0.2)

In [608]:
depthOfNetwork = 2
activationFuncEachLayer = ['sigmoid', 'relu', 'sigmoid']
neuronCountInEachLayer = [123, 65, 1]    
lossFunction = 'binary_crossentropy' 
regularizerFunc = tf.keras.regularizers.l2(0)

def createModel():
    model = tf.keras.models.Sequential()
    model.add(Dense(neuronCountInEachLayer[0], input_dim=8, activation = activationFuncEachLayer[0], kernel_regularizer=regularizerFunc))
    for i in range(depthOfNetwork - 1):
        model.add(Dense(neuronCountInEachLayer[i], activation=activationFuncEachLayer[i], kernel_regularizer=regularizerFunc))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss = lossFunction , optimizer = 'adam' , metrics = ['accuracy'] ) 
    return model

In [609]:
model = createModel()
model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_split=0.1)


Epoch 1/10
11/11 [==============================] - 1s 23ms/step - loss: 0.6919 - accuracy: 0.5146 - val_loss: 0.6945 - val_accuracy: 0.4416
Epoch 2/10
11/11 [==============================] - 0s 7ms/step - loss: 0.6900 - accuracy: 0.5395 - val_loss: 0.7209 - val_accuracy: 0.4416
Epoch 3/10
11/11 [==============================] - 0s 8ms/step - loss: 0.6932 - accuracy: 0.5395 - val_loss: 0.6915 - val_accuracy: 0.4416
Epoch 4/10
11/11 [==============================] - 0s 6ms/step - loss: 0.6911 - accuracy: 0.5599 - val_loss: 0.7158 - val_accuracy: 0.4416
Epoch 5/10
11/11 [==============================] - 0s 6ms/step - loss: 0.6878 - accuracy: 0.5409 - val_loss: 0.6836 - val_accuracy: 0.6494
Epoch 6/10
11/11 [==============================] - 0s 5ms/step - loss: 0.6871 - accuracy: 0.5512 - val_loss: 0.7029 - val_accuracy: 0.4416
Epoch 7/10
11/11 [==============================] - 0s 5ms/step - loss: 0.6882 - accuracy: 0.5424 - val_loss: 0.7053 - val_accuracy: 0.4416
Epoch 8/10
11/11 [=

In [610]:
test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f'Test accuracy: {test_acc * 100:.2f}%')

6/6 [==============================] - 0s 2ms/step - loss: 0.6903 - accuracy: 0.5079
Test accuracy: 50.79%
